In [2]:
import pandas as pd
import numpy as np
import re
import json
from datetime import timedelta, datetime

In [3]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 
from statistic import Statistic
from converter import Converter
pd.set_option('display.max_colwidth', -1)

In [4]:
# for autoreload modules
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
path_data = '/mnt/files/workdata/work/merchants/merchant_26_tutu2/01_raw_data/data-rail-with-id_2020-02-04/'

In [6]:
df = pd.read_csv(path_data + 'data-rail-with-id.csv', sep='\t',dtype=str,keep_default_na=False, nrows=100)

In [7]:
df.shape

(100, 5)

In [8]:
def is_problem_line(line):
    pattern = ':([a-zA-Z]+)'
    incorrect_values = {'true', 'false', 'null'}
    incorrect_values_in_line = set(re.findall(pattern, line))        
    diff = incorrect_values_in_line.difference(incorrect_values)
    return len(diff)

In [9]:
df['incorrect_line'] = df.json.apply(lambda x: is_problem_line(x))

In [10]:
Statistic.get_table(df, 'incorrect_line')

Counter({0: 100})

In [12]:
mask = (df.incorrect_line > 0)
df_incorrect_line = df[mask]

In [13]:
n = df_incorrect_line.shape[0]
for i in range(n):
    pattern = ':([a-zA-Z]+)'
    print(set(re.findall(pattern, df_incorrect_line.json.values[i])))

In [14]:
mask = (df.incorrect_line > 0)
df_correct_line = df[~mask]

In [15]:
df_correct_line.json.values[0] # true - не в двойный кавычках

'{"order":{"sms_notification":true,"is_round_trip":false,"has_rzd_bonus":false,"seats_selected":true,"site_zone":"main","is_composite_trip":false,"requested_fin_documents":false},"client":{"phone":"923143****","account_confirmed":true,"name":"Наталья Алтынбаева","account_exists":true,"email":"aaaaaa49@mail.ru"},"passengers":[{"sex":"F","has_middle_name ":true,"seat_level":"top","seat_side":false,"seat":"2","doc_type":"inner","birthday ":"1983-06-08","doc_country":"RUS"},{"doc_country":"RUS","birthday ":"2014-03-18","seat":"3","doc_type":"birth","seat_side":false,"seat_level":"bottom","has_middle_name ":true,"sex":"F"}],"train":{"arrival":"Абакан","is_trailer":false,"is_firm":false,"departure":"Москва Ярославская","departure_date":"2019-04-23 23:05:00"},"device":{"browser_version":null,"type":"android","http_headers":{"user_agent":"Mozilla/5.0 (Linux; Android 9; PAR-LX1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.119 YaBrowser/19.3.5.299.00 Mobile Safari/537.36"},"browser":

In [16]:
df_correct_line = df_correct_line.copy()
pattern = ':([a-zA-Z]+)'
df_correct_line['correct_json'] = df_correct_line.json.apply(lambda x: re.sub(pattern, ':"\\1"', str(x)))

In [17]:
df_correct_line.correct_json.values[0]

'{"order":{"sms_notification":"true","is_round_trip":"false","has_rzd_bonus":"false","seats_selected":"true","site_zone":"main","is_composite_trip":"false","requested_fin_documents":"false"},"client":{"phone":"923143****","account_confirmed":"true","name":"Наталья Алтынбаева","account_exists":"true","email":"aaaaaa49@mail.ru"},"passengers":[{"sex":"F","has_middle_name ":"true","seat_level":"top","seat_side":"false","seat":"2","doc_type":"inner","birthday ":"1983-06-08","doc_country":"RUS"},{"doc_country":"RUS","birthday ":"2014-03-18","seat":"3","doc_type":"birth","seat_side":"false","seat_level":"bottom","has_middle_name ":"true","sex":"F"}],"train":{"arrival":"Абакан","is_trailer":"false","is_firm":"false","departure":"Москва Ярославская","departure_date":"2019-04-23 23:05:00"},"device":{"browser_version":"null","type":"android","http_headers":{"user_agent":"Mozilla/5.0 (Linux; Android 9; PAR-LX1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.119 YaBrowser/19.3.5.299.00 Mob

In [18]:
df_correct_line.correct_json = df_correct_line.correct_json.apply(lambda x: re.sub(r'\\\\', r'\\', str(x)))

In [19]:
line = df_correct_line.correct_json.values[0]
line

'{"order":{"sms_notification":"true","is_round_trip":"false","has_rzd_bonus":"false","seats_selected":"true","site_zone":"main","is_composite_trip":"false","requested_fin_documents":"false"},"client":{"phone":"923143****","account_confirmed":"true","name":"Наталья Алтынбаева","account_exists":"true","email":"aaaaaa49@mail.ru"},"passengers":[{"sex":"F","has_middle_name ":"true","seat_level":"top","seat_side":"false","seat":"2","doc_type":"inner","birthday ":"1983-06-08","doc_country":"RUS"},{"doc_country":"RUS","birthday ":"2014-03-18","seat":"3","doc_type":"birth","seat_side":"false","seat_level":"bottom","has_middle_name ":"true","sex":"F"}],"train":{"arrival":"Абакан","is_trailer":"false","is_firm":"false","departure":"Москва Ярославская","departure_date":"2019-04-23 23:05:00"},"device":{"browser_version":"null","type":"android","http_headers":{"user_agent":"Mozilla/5.0 (Linux; Android 9; PAR-LX1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.119 YaBrowser/19.3.5.299.00 Mob

In [20]:
d = json.loads(line)
flat_d = Converter.get_flat_dictionary(d)
flat_d

{'order.sms_notification': 'true',
 'order.is_round_trip': 'false',
 'order.has_rzd_bonus': 'false',
 'order.seats_selected': 'true',
 'order.site_zone': 'main',
 'order.is_composite_trip': 'false',
 'order.requested_fin_documents': 'false',
 'client.phone': '923143****',
 'client.account_confirmed': 'true',
 'client.name': 'Наталья Алтынбаева',
 'client.account_exists': 'true',
 'client.email': 'aaaaaa49@mail.ru',
 'passengers.1.sex': 'F',
 'passengers.1.has_middle_name ': 'true',
 'passengers.1.seat_level': 'bottom',
 'passengers.1.seat_side': 'false',
 'passengers.1.seat': '3',
 'passengers.1.doc_type': 'birth',
 'passengers.1.birthday ': '2014-03-18',
 'passengers.1.doc_country': 'RUS',
 'train.arrival': 'Абакан',
 'train.is_trailer': 'false',
 'train.is_firm': 'false',
 'train.departure': 'Москва Ярославская',
 'train.departure_date': '2019-04-23 23:05:00',
 'device.browser_version': 'null',
 'device.type': 'android',
 'device.http_headers.user_agent': 'Mozilla/5.0 (Linux; Android

In [21]:
list(df_correct_line)

['order_id',
 'title',
 'merchant_order_id',
 'json',
 'created',
 'incorrect_line',
 'correct_json']

In [22]:
col_names = Converter.get_flat_dictionary_col_names(df_correct_line, 'correct_json')
col_names

col num =81
NULL


['trip.infants',
 'passengers.1.sex',
 'device.browser',
 'passengers.1.doc_country',
 'train.is_trailer',
 'currency',
 'trip.car_category',
 'trip.days_to_departure',
 'trip.arrival_date',
 'trip.registration_available',
 'device.http_headers.user_agent',
 'amount',
 'location.ip',
 'trip.travel_duration',
 'order.requested_fin_documents',
 'order.site_zone',
 'trip.include_laundry',
 'date',
 'train.arrival',
 'train.is_firm',
 'client.account_exists',
 'trip.adults',
 'trip.departure',
 'passengers.1.has_middle_name ',
 'trip.departure_latitude',
 'client.phone',
 'passengers.1.seat',
 'device.type',
 'trip.passengers',
 'client.name',
 'order.has_rzd_bonus',
 'trip.arrival_longitude',
 'trip.children',
 'order.seats_selected',
 'client.email',
 'order.sms_notification',
 'order.is_composite_trip',
 'trip.arrival',
 'trip.departure_country',
 'passengers.1.birthday ',
 'trip.insurance_selected',
 'train.departure_date',
 'client.account_confirmed',
 'passengers.1.seat_side',
 'trip

In [26]:
output_file = path_data + 'tmp.converted.csv'
Converter.write_json_column_to_csv(df_correct_line, 'correct_json', col_names, output_file)

col num =81
NULL


tmp.shape

In [24]:
col_name = 'passengers.1.sex'
col_json_name = 'correct_json'
col_pas_1 = Converter.convert_one_column_from_json(df_correct_line, col_json_name, col_name)

In [25]:
col_pas_1.values[:7]

array(['F', 'F', 'M', 'M', 'M', 'F', 'F'], dtype=object)